In [ ]:
import sys
sys.path.insert(0, '../')

import utils, evaluation
#from evaluation import load_gan
from data import fmap, Dataset
from JR_Scripts import dict_reader, time_toy_generator
from JR_Scripts import notebook_helper as nh
from model import WGanModel, WNGanModel, TemporalGanModelv3
from gan import TimeCosmoGAN
import metrics
import tensorflow as tf
import numpy as np
from PIL import Image
import imageio
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import importlib as imp

In [ ]:
redshift = ['z = 0.000', 'z = 0.111', 'z = 0.250', 'z = 0.428', 'z = 0.666', 'z = 1.000', 'z = 1.500', 'z = 2.333', 'z = 4.000', 'z = 9.000']

In [ ]:
def load_gan_and_gen_data_in_old_format(model_folder, specific_model, params):
    gan = evaluation.load_gan(model_folder, GANtype=TimeCosmoGAN)
    t = params['time']['classes']
    images = nh.gen_fake_images(gan, 5000, t, checkpoint=specific_model)
    return params['cosmology']['backward_map'](images)

In [ ]:
def modify_param_classes(params):
    lst_classes = []
    for cl in params['time']['classes']:
        lst_classes.append(cl + 1)
    params['time']['classes'] = lst_classes
    lst_cl_weights = []
    for cl in params['time']['class_weights']:
        lst_cl_weights.append(cl - 0.08)
    params['time']['class_weights'] = lst_cl_weights

In [ ]:
data_lst = []
labels_lst = ['Encode Channel', 'Scale Half', 'Scale Full', 'Real']
colors_lst = ['b', 'g', 'y', 'r']

In [ ]:
params, model_folder = nh.load_params("models_64x64_C4/TWGANce:62d3_selu_sn10-10_0246r_Hlr3e5_bs4_v2ad_c+sf62_checkpoints/", cscs=True)
specific_model = 164800

In [ ]:
modify_param_classes(params)
nh.print_param_dict(params)

In [ ]:
data_lst.append(load_gan_and_gen_data_in_old_format(model_folder, specific_model, params))

In [ ]:
params, model_folder = nh.load_params("models_64x64_C4/TWGANsh:62d3_selu_sn10-10_0246r_Hlr3e5_v2ad_c+sf62_checkpoints/", cscs=True)
specific_model = 142400

In [ ]:
modify_param_classes(params)
nh.print_param_dict(params)

In [ ]:
data_lst.append(load_gan_and_gen_data_in_old_format(model_folder, specific_model, params))

In [ ]:
params, model_folder = nh.load_params("models_64x64_C4/TWGANsf:62d3_selu_sn10-10_0246r_Hlr3e5_v2ad_c+sf62_checkpoints/", cscs=True)
specific_model = 132800

In [ ]:
modify_param_classes(params)
nh.print_param_dict(params)

In [ ]:
data_lst.append(load_gan_and_gen_data_in_old_format(model_folder, specific_model, params))

In [ ]:
dset = nh.get_dataset(500, 512, params, shuffle=True, cscs=True)

In [ ]:
#nh.get_scaling(gan.params['time']['classes'], gan.params['time']['class_weights'])

In [ ]:
data = dset.get_samples(5000)
data = nh.reshape_data_to_old_format(data, params)
data = params['cosmology']['backward_map'](data)
data.shape

In [ ]:
data_lst.append(data)

# Single Timestep Data Analysis

## Mass Histograms

### Progression over time

In [ ]:
#imp.reload(nh)

### Real-Fake timewise comparisons

In [ ]:
lim_mass = nh.get_lim_mass(data)
nh.plot_mass_hists(data_lst, labels_lst, colors_lst, lim_mass, params)

## Peak Histograms

### Progression over time

In [ ]:
_, _, lim = metrics.peak_count_hist(data_lst[1])
lim = (lim[0], lim[1] + 0.5)
#nh.peak_hist_over_time(data, images, params, lim=lim, title_a="Real Data", title_b="Fake Data")

### Real-Fake timewise comparison

In [ ]:
nh.plot_peak_hists(data_lst, labels_lst, colors_lst, lim, params)

## Power Spectral Densities

### Progression over time

In [ ]:
#nh.power_spectral_densities(data, params, data_name="Real Data")
#nh.power_spectral_densities(images, params, data_name="Fake Data")

In [ ]:
imp.reload(nh)

### Real-Fake timewise comparison

In [ ]:
nh.plot_power_spectral_densities(data_lst, labels_lst, colors_lst, params)

### Mass Diff Histograms

In [ ]:
imp.reload(nh)

In [ ]:
nc = 4
max_lim_pos = 0
max_lim_neg = 0
for i in range(nc-1):
    lim_mass = nh.get_lim_mass(np.clip(data[(i+1)::nc]-data[i::nc], a_min=0, a_max=None))
    if lim_mass[1] > max_lim_pos:
        max_lim_pos = lim_mass[1]
    lim_mass = nh.get_lim_mass(np.clip(data[i::nc]-data[(i+1)::nc], a_min=0, a_max=None))
    if lim_mass[1] > max_lim_neg:
        max_lim_neg = lim_mass[1]
lim_mass = (max_lim_pos, max_lim_neg)
print(lim_mass)
nh.plot_mass_hist_diffs(data_lst, labels_lst, colors_lst, lim_mass, params)

In [ ]:
nh.get_scaling(params['time']['classes'], params['time']['class_weights'])